In [1]:
import numpy as np
import pandas as pd
import os
from hmmlearn.hmm import GaussianHMM
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split

In [2]:
os.chdir('C:\\Users\\grace\\UNSW\\COMP4121\\COMP4121_MajorProject\\Data')

In [3]:
# Load the saved values from the .npz file
data = np.load('hmm_data.npz')

In [ ]:
# Retrieve the variables
n_states = int(data['n_states'])
features = data['X']

print(f"Optimal number of hidden states: {n_states}")
print(f"Features shape: {features.shape}")

Optimal number of hidden states: 5
Features shape: (244, 2)


In [14]:
merged_df = pd.read_csv('merged_df.csv')
print(merged_df.head())

     datesold    price        MA  bedrooms  propertyType  day_of_week  \
0  2007-06-30   517000  548969.0         4             0            5   
1  2008-06-30   480000  441854.0         2             0            0   
2  2008-06-30  1165000  735904.0         5             0            0   
3  2008-09-30   312000  377481.0         2             1            1   
4  2008-09-30   367500  561852.0         4             0            1   

   day_of_month  seasonality  Regime  hidden_state  
0             0            2       0             0  
1             0            2       1             1  
2             0            2       3             3  
3             0            2       1             1  
4             0            2       0             0  


Split Data into Train and Test Sets

In [15]:
# Use the price column for ARIMA
train, test = train_test_split(merged_df[['datesold', 'price']], test_size=0.2, shuffle=False)

# Ensure the data is indexed by date for ARIMA
train.set_index('datesold', inplace=True)
test.set_index('datesold', inplace=True)

Fit ARIMA Model

In [16]:
# Fit ARIMA model (example order, adjust as necessary)
arima_model = ARIMA(train['price'], order=(1, 1, 1))
arima_fit = arima_model.fit()

# Forecast on the test set
forecast = arima_fit.forecast(steps=len(test))


c:\Users\grace\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\grace\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\grace\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\grace\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is ava

In [17]:
# Calculate residuals
residuals = train['price'] - arima_fit.fittedvalues


In [18]:
train['residuals'] = residuals
train['hidden_state'] = merged_df.loc[train.index, 'hidden_state']

# Analyze residuals by hidden state
residuals_by_state = train.groupby('hidden_state')['residuals'].mean()
print(residuals_by_state)


KeyError: "None of [Index(['2007-06-30', '2008-06-30', '2008-06-30', '2008-09-30', '2008-09-30',\n       '2008-09-30', '2008-09-30', '2008-09-30', '2008-09-30', '2008-09-30',\n       ...\n       '2016-09-30', '2016-09-30', '2016-09-30', '2016-09-30', '2017-03-31',\n       '2017-03-31', '2017-03-31', '2017-03-31', '2017-03-31', '2017-03-31'],\n      dtype='object', name='datesold', length=195)] are in the [index]"

Incorporate Hidden States as Features in ARIMA:

Add the hidden states as an exogenous variable to ARIMA. This allows ARIMA to condition forecasts on the regimes identified by the HMM.

In [ ]:
# Use hidden states as an exogenous variable
exog_train = pd.get_dummies(train['hidden_state'], prefix='state')
exog_test = pd.get_dummies(test['hidden_state'], prefix='state')

# Fit ARIMA with exogenous variables
arima_exog_model = ARIMA(train['price'], exog=exog_train, order=(1, 1, 1))
arima_exog_fit = arima_exog_model.fit()

# Forecast with exogenous variables
forecast_exog = arima_exog_fit.forecast(steps=len(test), exog=exog_test)
